# Example: Local field potential

In this example, we show how one can sample the Local Field Potential around a neuron, by extracting the necessary data from the simulation and applying the LFP caclulations for the desired sample points.

If one adds a population of neurons, does the relevant changes and focuses the sample points closer, the same methods can also yield the multi-unit activity (mind the simulation timestep though).

---
First of all, let's install the required software, if on certain platforms like Colab that run the bare notebooks:

In [ ]:
import os; from pathlib import Path
if 'COLAB_BACKEND_VERSION' in os.environ:
  !TMP=$(mktemp -d); git clone https://eden-simulator.org/repo --depth 1 -b development "$TMP"; cp -r "$TMP/." .; rm -rf "$TMP"
  exec(Path('.binder/install_livenb.py').read_text())
if 'DEEPNOTE_PROJECT_ID' in os.environ: exec(Path('../.binder/install_livenb.py').read_text())

---

## Get the model
Let's see what the LFP looks like for the L5 pyramidal cell from [Mainen et al. 1995]( https://github.com/OpenSourceBrain/MainenEtAl_PyramidalCell ):

In [ ]:
# Download the zip file with the model
import urllib.request
urllib.request.urlretrieve('https://codeload.github.com/OpenSourceBrain/MainenEtAl_PyramidalCell/zip/refs/heads/master', 'Mainen95.zip')
# and unpack it
from zipfile import ZipFile
with ZipFile('Mainen95.zip', 'r') as zipp: zipp.extractall()
# Check which files are present now
# from os import listdir; listdir()

Open the folder (through the GUI or os.listdir) and locate the NeuroMLv2 folder.

In [ ]:
nml_dir = 'MainenEtAl_PyramidalCell-master/neuroConstruct/generatedNeuroML2/'
# listdir(nml_dir)

## Prepare and run the simulation

We'll run the existing setup with a lone neuron and a pulse clamp on it, but we'll make a whole new simulation file to capture all points of the neuron.

First analyse the cell to extract the compartments this neurons contains:

In [ ]:
net_filename = nml_dir+"MainenEtAl_PyramidalCell.net.nml"
celltype_name = 'MainenNeuroML'
import eden_simulator
cells_info = eden_simulator.experimental.explain_cell(net_filename)
cell_info = cells_info[celltype_name]

In [ ]:
comp_mid_seg = cell_info['comp_midpoint_segment']
comp_mid_fra = cell_info['comp_midpoint_fractionAlong']
n_comps = len(comp_mid_seg) # Number of actual compartments in this cell, may be retrieved from most cell_info arrays
print(f'Compartments: {n_comps}')

And then generate the new simulation file with its traces:

In [ ]:
network_name = 'network_MainenEtAl_PyramidalCell';
population_name = 'NeuroMLBased'; N_cells = 1
# Set a new routine for making the <Simulation> file with new recording options
def NmlSimParms(network_name, run_time, run_timestep=25e-6, sampling_period=1e-3,
            rec_lines='', href='file://results.gen.txt'):
    return f'''
        <Simulation id="sim1" length="{run_time}s" step="{run_timestep}s" target="{network_name}">
            <EdenOutputFile id="first" href="{href}" format="ascii_v0" sampling_interval="{sampling_period} s">
            '''+'\n\t        '.join(rec_lines)+'''
            </EdenOutputFile>
        </Simulation>
        <Target component="sim1"/>'''

# Make the traces to record each compartment, and save the file
traces = [f"{population_name}[{neu}]/{celltype_name}/{comp_mid_seg[i]}{('%.9f'%comp_mid_fra[i])[1:]}/v"
    for neu in range(N_cells) for i in range(len(comp_mid_seg))]
rec_lines = [f'<OutputColumn id="v_{i}" quantity="{x}"  output_units="mV"/>' for i,x in enumerate(traces) ]
print(f"recording {len(rec_lines)} waveforms this time !")
sim_filename = nml_dir+"/SimLFP.xml"
with open(sim_filename, "w") as f:   
    f.write('<?xml version="1.0" encoding="UTF-8"?>\n<Lems>\n<include file="MainenEtAl_PyramidalCell.net.nml"/>\n')
    f.write(NmlSimParms(network_name, run_time=0.06, run_timestep=10e-6, sampling_period=0.1e-3, href='moresults.gen.txt', rec_lines=rec_lines))
    f.write('\n</Lems>\n')
# !cat $nml_dir/SimMore.xml

And run the simulation:

In [ ]:
%time moresults = eden_simulator.runEden(sim_filename)
timevec = moresults['t']

Since we'll be doing physics with the simulation's results so that we can calculate the LFP, let's convert the $V_m$ data to a physical `Quantity` with the help of the `pint` [Python package]( https://pint.readthedocs.io/en/stable/user/defining-quantities.html ) for unit math.

In [ ]:
!pip install -q pint

In [ ]:
import numpy as np
from pint.registry import Quantity as Q
neuron_membrane_voltage = np.array([moresults[x] for i,x in enumerate(traces)])
neuron_membrane_voltage = Q(neuron_membrane_voltage,'mV')
print("%d points in space, sampled over %d points in time." % neuron_membrane_voltage.shape)

Let's see a raster of $V_m$ per compartment: 

In [ ]:
from matplotlib import pyplot as plt
im = plt.imshow(neuron_membrane_voltage.m_as('mV'),
    extent=[ timevec[0], timevec[-1], n_comps, 0 ],
    aspect='auto', interpolation='none', cmap ="viridis")
cbar = plt.colorbar(im); cbar.set_label('Voltage (mV)')
plt.clim((-50, +50)); plt.ylabel('Compartment #'); plt.xlabel('Time (sec)');

And an animation of said $V_m$:

In [ ]:
from eden_simulator.display.animation import subsample_trajectories
samples_vm, anim_axis_vm, sampled_time_vm, (sampled_voltage,) = subsample_trajectories(
    timevec, [neuron_membrane_voltage.m_as('mV').T], animation_speed=0.0048, animation_frames_per_second=24)

import k3d; from eden_simulator.display.spatial.k3d import Plot, plot_neuron
Vm_plot = plot = Plot()
plot += plot_neuron(cell_info, sampled_voltage, time_axis_sec=anim_axis_vm,
                             color_range=[-70, -20],color_map='rainbow');
plot.camera = plot.get_auto_camera(pitch=30, yaw=10)[:6]+[0,1,0] # set 'y' to up ! vecs are pos, tgt, up
k3d_label_dict = { str(real_time): f't = {sim_time*1000:.1f} ~ ms' for (real_time, sim_time) in zip(anim_axis_vm, sampled_time_vm)  }
plt_label = k3d.text2d(k3d_label_dict, (0.,0.)); plot += plt_label # add 2d elements AFTER setting auto camera
plot.show_html()

## Post-process simulation data to get $I_m$

We have recorded $V_m$ all over the cell and we'll use it (along with the cell's properties) to obtain the resulting LFP.

The usual assumptions apply for this example: The extracellular medium is uniform, highly conductive, isotropic and unobstructed until far away.  If you need a more sophisticated model, apply the relevant adjustments to the following.  (Perhaps a separate simulation process just for the LFP is called for, see the other articles TBD on "Pipelines" generally.)
<!-- LATER pipelines -->

If storing all voltage traces is becoming a burden, one can calculate the coupling matrix as they prefer and add up the numbers using virtual "gap junctions".  (The virtual gap junction method can be shown in a TBA article).  
For really large numbers of current sources, it might become more efficient to stream values with a pipeline to the Barnes-Hut algorithm or the fast multipole method.

In this tutorial we'll be looking at the LFP, hence we need $I_m$ .  Instead of adding up the current from each mechanism, we can estimate it through $dV_m/dt$. To reduce sampling artifacts, we could convolve with a [derivative of Gaussian]( https://inst.eecs.berkeley.edu/~cs194-26/fa17/Lectures/ConvEdgesTemplate.pdf ) in the place of `np.diff` but it's not a concern here.

The dynamics of membrane voltage per compartment, are as follows: 
$$C\dot{V}_m = I_{axial} + I_m + I_{injected}$$ 
Therefore we should subtract the current from adjacent compartments and the current from direct injection through the membrane, to isolate the trans-membrane current.

*Note:* Although direct injections cross the membrane (topologically), they don't count towards the LFP because charge travels from inside a body to inside another; thus the extracellular environment is not affected.  
Examples of direct injections are current clamps, as well as gap junctions between cells.

Whether a stimulus from our rig should count as trans-membrane current or direct injection is ultimately up to the modeller's intention; due caution is advised. (eg. is a "virtual synapse" stimulus imposed by a clamp, or a simulated synapse?)

In [ ]:
# J = dQ/dt = C * dV/dt
rec_dt = Q(np.diff(timevec)[0],'sec')
comp_capa = Q(cell_info['comp_capacitance'],'pF')

neuron_comp_flux = comp_capa[:,None]*(np.diff(neuron_membrane_voltage)/rec_dt)
neuron_comp_flux.ito('pA') # Convert to specific units this way

Let's see the total current flow for each compartment.

The simulator provides absolute numbers for unevenly sized compartments, the *density* over the membrane is perhaps more informative:

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(18,5))
im = axs[0].imshow(neuron_comp_flux.m_as('pA'),
    extent=[ moresults['t'][0], timevec[-1], n_comps, 0 ],
    aspect='auto', interpolation='none', cmap ="bwr_r");
cbar = plt.colorbar(im); cbar.set_label('pA')
im.set_clim((-30, +30)); axs[0].set_ylabel('Compartment #'); axs[0].set_xlabel('Time (sec)');
axs[0].set_title("Comp. current")
# Now, get the density
comp_area = Q(cell_info['comp_area'], 'um*um')
neuron_comp_flux_density = neuron_comp_flux/comp_area[:,None]
im = axs[1].imshow(neuron_comp_flux_density.m_as('pA/um**2'),
    extent=[ moresults['t'][0], timevec[-1], n_comps, 0 ],
    aspect='auto', interpolation='none', cmap ="bwr_r");
cbar = plt.colorbar(im); cbar.set_label('pA/μm²')
im.set_clim((-.4, +.4)); axs[1].set_ylabel('Compartment #'); axs[1].set_xlabel('Time (sec)');
axs[1].set_title("Current density");

This is the total current flux in/out of the compartment, where `+` is for current `in` and `-` for current `out` of the cell (of course, the reverse applies for electrons and negative ions).  
We use the `bwr_r` colormap where blue is for positive flow `in`ward, because this flow will cause an opposite, *negative* potential outside the cell - which we'll get to see in the calculated LFP.

To correct the numbers, let's calculate and subtract the "axial" current flowing between compartments, based on the known electrical conductance between them.  
In this example, axial current is far far less than true trans-membrane current so there's no obvious difference, we should do it nonetheless.

### Correcting for axial current

In [ ]:
# First, get the axial resistance matrix: Ia = G*V. It follows the tree structure
comp_ga = cell_info['comp_conductance_to_parent']
iii = []; jjj = []; vvv = [] # Construct the lists (i,j,v)
for i,p in enumerate(cell_info['comp_parent']):
    g = comp_ga[i]
    if p >= 0:
        iii += [ i, p, i, p] # add 4 sparse elms in one go
        jjj += [ p, i, i, p]
        vvv += [+g,+g,-g,-g] # same as:
        # axial_conductance_matrix[i,p] += g
        # axial_conductance_matrix[p,i] += g
        # axial_conductance_matrix[i,i] -= g
        # axial_conductance_matrix[p,p] -= g

import scipy
G = scipy.sparse.coo_matrix((vvv,(iii,jjj))).tocsr() # needs csr for some reason'
G = Q(G,'nS')

# One would think Ia = G*V directly, but the discretisation of the neuron 
# leads to rough ΔV gradient between compartments in practice (plot Vm of the stylized axon, last 125 comps, around sample 100 to see). 
# -> hence Ia would come out extremely rough and inaccurate.
# In its place we will use Bwd Euler to estimate the currents (neglecting ion channel conductance tbh):
# Ia = C*dV/dt = G*V => Ia*dt = Vnext-Vnow = dt*inv(C)*G*Vnext, Vnext = inv(I-dt*inv(C)*G)*Vnow.
# Then, make the dimensionless "dynamics matrix" that will give us Vnext from vnow.

I = scipy.sparse.eye(n_comps); invC = (scipy.sparse.diags(1/comp_capa.m))
# neither pint nor quantities can do matmul :c
dynamics_matrix = I - (
        (rec_dt.units*G.units/comp_capa.units) * (rec_dt.m*invC @ G.m)
    ).to('1').m # cast to proper quantity, not eg. meters/km


def get_true_membrane_current(comp_flux, membrane_voltage, G, time=0): 
    # instead of direct: axial_current = (G.m @ membrane_voltage.m)*G.units*membrane_voltage.units
    vNext_axial = Q(scipy.sparse.linalg.spsolve(dynamics_matrix, membrane_voltage.m_as('mV')),'mV')
    axial_current = (vNext_axial -  membrane_voltage) * comp_capa[:,None] / rec_dt
    return (comp_flux - axial_current)
neuron_membrane_current = get_true_membrane_current(neuron_comp_flux, neuron_membrane_voltage[:,:-1], G)

**NOTE**: because direct currents such as patch clamps come from insulated wires, they *don't* affect the observed LFP, hence it is well possible that a 'monopole' appears during such stimulation!

Kirchhoff's law still stands: $$I_{membrane} = -(I_{capacitive} + I_{injected})$$
it's just that we can't measure the right hand side through extracellular space.

If you're interested, repeat the graph from above to plot the difference between `neuron_comp_flux` and `neuron_membrane_current`, in absolute and per area terms.

In [ ]:
# intentionally left blank 

Observe that come compartments are much more affected by axial current than others.  
Generally, the less area they have, and the tighter they are coupled to adjacent compartments, the more affected they are by axial current. (Cytoplasmic resistivity `Ra` and membrane spec. capacitance `Cm` can also matter when non-uniform.)  
Feel free to plot `comp_ga` and `comp_capa` (and the ratio thereof `1/RC`) to see which these may be.

### Visualisation
Let's see then how the trans-membrane current evolves on the neuron.  We'll use the same Red-White-Blue colormap we'll display the LFP with, in the following.

In [ ]:
# resample the anim_axis because Im has 1 time point less than Vm, due to np.diff
samples_im, anim_axis_im, sampled_time_im, [sampled_current_density] = subsample_trajectories(
    timevec[:-1], [(neuron_membrane_current / comp_area[:,None]).m_as('pA/um**2').T], animation_speed=0.0048, animation_frames_per_second=24)

Im_plot = plot = Plot()
plot += plot_neuron(cell_info, sampled_current_density, time_axis_sec=anim_axis_im,
                             color_range=[-.2, +.2],color_map='bwr');
plot.camera = plot.get_auto_camera(pitch=30, yaw=10)[:6]+[0,1,0] # set 'y' to up ! vecs are pos, tgt, up
plot.show_html()

## Post-process $I_m$ to get the LFP

The other half of calculating the LFP is how much each point outside the cell is being affected by the current flow on each part of the neuron (that is, mostly by the closest compartments, but also slightly from the more distant ones).

Just for illustration, we'll use one element per compartment. It would be even better if each NeuroML `<segment>` is applied separately, if it's not too much for the computer.  
Alternatively, the [LFPykit]( https://github.com/LFPy/LFPykit ) could also be used to calculate the coupling matrix.

In [ ]:
extracell_conductivity = Q(.3, 'S/m')

source_points = cell_info['comp_midpoint'][:]
x_space = np.linspace(-360, 000, 10); y_space = np.linspace(-180, 600, 30); z_space = [-30]
x, y, z = np.meshgrid(x_space, y_space, z_space, indexing='ij')
sampling_points = np.stack((x,y,z),axis=-1).reshape((-1,3))
# print("Sampling points:", sampling_points)

# Vectorize calculation of pairwise distance matrix https://jaykmody.com/blog/distance-matrices-with-numpy/
distmat =  source_points @ sampling_points.T#np.sum(,axis=-1)
distmat = np.sum(sampling_points**2,axis=-1) - 2*distmat + np.sum(source_points**2,axis=-1)[:,None]
distmat = np.sqrt(distmat)
distmat = Q(distmat, 'um')

# LATER: Exclude current from points inside the cell, thereby avoiding singularities as a bonus.
coupling = (1e-0 / (4*np.pi*extracell_conductivity*distmat)).to('uV/pA')

print('Coupling is calculated for %d current sources times %d sampling points.' % coupling.shape)
def get_lfp(membrane_current): return ((coupling.m.T @ -membrane_current.m)*coupling.units*membrane_current.units).to('uV')
# get_lfp(neuron_membrane_current[:,0])

Now let's show the membrane voltage and the observed LFP together in one animation.

In [ ]:
# Animate the lfp, similar to label but with a vector instead of a string
lfp_uvolt = get_lfp(neuron_membrane_current).m_as('uV').T
k3d_lfp_dict = { str(real_time): x.astype(np.float32) 
                for (real_time, x) in zip(anim_axis_vm, lfp_uvolt[samples_im])  }

In [ ]:
LFP_plot = plot = Plot()

lfp_scale_range = [-2, 2] #other options to see transitions vs. absolute range: [-.2, .2], [-.5, +.5] ... 
plt_points = k3d.points(positions=sampling_points.astype(np.float32),attribute=k3d_lfp_dict,
        color_range=lfp_scale_range,point_size=10,color_map=k3d.matplotlib_color_maps.Bwr); plot += plt_points

plot.camera = plot.get_auto_camera(pitch=30, yaw=10)[:6]+[0,1,0] # set 'y' to up ! vecs are pos, tgt, up
plt_mesh = plot_neuron(cell_info, sampled_voltage, time_axis_sec=anim_axis_vm,
                    color_range=[-70, -20],color_map='rainbow', compress_cells=False);
plot += plt_mesh
plt_label = k3d.text2d(k3d_label_dict, (0.,0.)); plot += plt_label # add 2d elements AFTER setting auto camera
plot.show_html()

The dots around the neuron are the sampling points for the LFP, they don't necessarily have to be a regular grid (but it's easier to understand this way).  
They are coloured with red ❤️ for positive potential, blue 💙 for negative, and white 🤍 for zero.

Observe that the LFP goes negative (blue) near where the cell depolarizes, and positive (red) where the cell repolarizes. (Also depnding on the activity of the more distant parts, of course.)

### More ideas

The enterprising reader may find more nuances and improved ways of calculating the LFP in the work of, among others, Einevoll and his lab. 

For best results, consider using non-uniform sampling grids, centered around where the potential changes most sharply and where the (both stimulation and recording) electrodes are located.

---

Now as the last thing, we'll fetch a screenshot to use in the documentation's [example gallery]( https://eden-simulator.org/gallery.html ).  

In [ ]:
plot = LFP_plot
plot.grid_visible = False
plot.screenshot_scale = 1
plot.axes_helper = 0
plot.colorbar_object_id = 0 # Disabling colorLegend programmatically
plt_points.color_range = [-1, +1]
plt_points.point_size = 15
plt_label.visible=False
plot -= plt_label
data_timestep = np.diff(timevec)[0] # assuming fixed timestep
traj_samples_per_frame = round((0.0048/24)/data_timestep)
# print(len(list(k3d_anim_dict.keys())))
frameis = [x for x in range(int(0.031/(data_timestep*traj_samples_per_frame)),int(0.031/(data_timestep*traj_samples_per_frame))+1,traj_samples_per_frame)]
frames = []
# for frame,sample in enumerate(sample_for_frame):print(sample_for_frame)
# print(framets)

# print(plt_mesh.attribute)
# raise
import time
try:
    from k3d.headless import k3d_remote, get_headless_driver
    headless = k3d_remote(plot, get_headless_driver(), width=500, height=300)
#     # plot.camera = lotp.get_auto_camera(pitch=30, yaw=10)[:6]+[0,1,0] # set 'y' to up !  LATER zoom a bit also; vecs are pos, tgt, up
    plot.camera = [0, 0, +200, 0,0,0, -1,0,0]
    headless.sync(hold_until_refreshed=True)
    headless.camera_reset(.49)
    # Set each attribute because does headless not like dicts ...
    for frame,i in enumerate(frameis):
        # need to clear before reassigning...
        attt = list(plt_mesh.attribute.items())[i][1]
        for x in plt_points,plt_mesh: x.attribute = {k:[] for k,v in x.attribute.items()}
        
        plt_mesh.attribute = attt
        plt_points.attribute = list(k3d_lfp_dict.items())[i][1]

        headless.sync()
        time_start = time.time()
        screenhot = headless.get_screenshot()
        time_end = time.time()
        # print(f"{time_end - time_start} sec")
        frames.append(screenhot)
finally:
    headless.close()
import IPython
IPython.display.Image(data=frames[-1])

In [ ]:
from PIL import Image
import PIL
import io

# Stage 1: Crop
frame_data = [ Image.open(io.BytesIO(x)) for x in frames]
frame_data = [ x.crop((int(0.3*x.size[0]), int(0.2*x.size[1]), int((1-.01)*x.size[0]), int((1-0.1)*x.size[1]))) for x in frame_data ]
# frame_data = [ x.crop((int(0.25*x.size[0]), int(0.25*x.size[1]), x.size[0], x.size[1])) for x in frame_data ]

# Stage 2: Resize
for x in frame_data: x.thumbnail((240,200))
    
# Stage 3: Lossy - or not
# frame_data = [x.convert('P',palette=Image.Palette.ADAPTIVE,dither=Image.Dither.NONE,colors=256) for x in frame_data]
# to apng, or gif
outbuf = io.BytesIO()
frame_data[0].save(outbuf, format='gif', save_all=True, append_images=frame_data[1:], duration=100, loop=0)
print('GIF size:',len(outbuf.getvalue()))

# to alpha channel out - how is this not compressed already
im = Image.open(outbuf)
# https://github.com/python-pillow/Pillow/issues/3292#issuecomment-410837926
newframes = [x.copy().convert('RGB') for x in PIL.ImageSequence.Iterator(im)]

aabuf = io.BytesIO()
newframes[0].save(aabuf, format='png', save_all=True, append_images=newframes[1:], duration=100, loop=0)
print(len(aabuf.getvalue()))

# to lossless recompression
import oxipng
oxipng_opts = {'level':6}
aa = oxipng.optimize_from_memory(aabuf.getvalue(), **oxipng_opts)
print('Optimized PNG size:',len(aa))

# display(IPython.display.Image(data=outbuf.getvalue()))
# display(IPython.display.Image(data=aabuf.getvalue()))
display(IPython.display.Image(data=aa))

# IPython refuses to nbconvert gif files, and apng is not as efficient, we'll have to get creative ...
# Accept having a png file for now? nah it's twice as big as the gif after gif compression.
# and save the file under an explicit name bc nbsphinx is misbehaving
with open('_static/thumb_exa_lfp.png','wb') as f: f.write(aa)
# with open('_static/thumb_tut_net.gif','wb') as f: f.write(outbuf.getvalue())

And minimize plots for publishing.

In [ ]:
from eden_simulator.display.spatial.k3d import MinimizePlot
for x in [Vm_plot, Im_plot, LFP_plot]: MinimizePlot(x)